

- `torch`：用于加载和处理PyTorch张量。
- `BertTokenizer` 和 `BertForSequenceClassification`：从Hugging Face的Transformers库中导入，用于加载BERT分词器和BERT模型。
- `Trainer` 和 `TrainingArguments`：用于训练模型的Trainer API。
- `load_dataset` 和 `load_metric`：从Hugging Face的`datasets`库中导入，用于加载数据集和计算评估指标。
- `np`：用于处理数组的NumPy库。
- `warnings`：用于控制警告信息。




In [1]:
import os

# 获取当前工作目录
current_directory = os.getcwd()
print(f"Current directory before change: {current_directory}")

# 要更改的目标目录
target_directory = 'Are-You-Mad?'

# 更改当前工作目录
os.chdir(target_directory)

# 获取更改后的当前工作目录地址
new_directory = os.getcwd()
print(f"Current directory after change: {new_directory}")


Current directory before change: /teamspace/studios/this_studio
Current directory after change: /teamspace/studios/this_studio/Are-You-Mad?


In [1]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, load_metric
import numpy as np
import warnings

# 忽略所有警告
warnings.filterwarnings('ignore')

# 加载IMDb电影评论数据集
dataset = load_dataset('imdb')

# 查看数据集信息
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


### 数据集结构

输出结果如下：

```plaintext
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})
```

- `DatasetDict`：这是一个字典，包含数据集的多个部分（分割）。
- `train`：训练集，包含25,000条电影评论。
    - `features`：特征包括`text`（电影评论文本）和`label`（情感标签）。
    - `num_rows`：样本数量。
- `test`：测试集，包含25,000条电影评论。
    - `features`：特征包括`text`（电影评论文本）和`label`（情感标签）。
    - `num_rows`：样本数量。
- `unsupervised`：无监督数据集，包含50,000条电影评论。这部分数据通常用于无监督学习任务。

### 解释

- **训练集（train）**：用于训练模型，包含电影评论的文本和对应的情感标签。
- **测试集（test）**：用于评估模型的性能，包含电影评论的文本和对应的情感标签。
- **无监督数据集（unsupervised）**：包含未标注的数据，可以用于无监督学习任务，例如词嵌入训练或自监督学习。

---

In [2]:

# 检查数据集的字段和示例
print(dataset['train'].features)

# 查看训练集中的前5个样本
for i in range(5):
    print(f"Sample {i+1}:")
    print(f"Text: {dataset['train'][i]['text']}")
    print(f"Label: {dataset['train'][i]['label']}")
    print()


{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}
Sample 1:
Text: I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 4


### 输出解释

#### 数据集的特征信息

```plaintext
{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}
```

- `text`：这是数据集中的文本字段。
  - **Value(dtype='string', id=None)**：
    - `dtype='string'`：数据类型是字符串。
    - `id=None`：没有指定特定的ID。
  - **解释**：每条数据样本包含一个文本字段，数据类型是字符串。

- `label`：这是数据集中的标签字段。
  - **ClassLabel(names=['neg', 'pos'], id=None)**：
    - `names=['neg', 'pos']`：标签包含两个类别，即负面（neg）和正面（pos）。
    - `id=None`：没有指定特定的ID。
  - **解释**：每条数据样本包含一个标签字段，标签可以是负面或正面。

#### 样本详细信息

```plaintext
Sample 1:
Text: I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. ......
Label: 0
```

- **Sample 1**：
  - **Text**：这是一条电影评论。评论中提到电影《I AM CURIOUS-YELLOW》的剧情、历史背景及其在1967年首次发布时的争议。
  - **HTML标签**：文本中包含多个`<br />`标签，表示换行。
  - **Label**：标签为0，表示这是一个负面评论。

### 解释

- **文本字段**：
  - **内容**：包含电影评论的文本，长度不一，内容丰富。
  - **HTML标签**：文本中可能包含HTML标签，这些标签需要在数据清理步骤中移除，以确保文本数据的干净和一致性。

- **标签字段**：
  - **负面标签（Label: 0）**：表示这条评论是负面的。
  - **正面标签**：如果标签为1，则表示这条评论是正面的。



我们看见数据集包含两个字段：text和label。text字段包含电影评论文本，label字段包含情感标签。

但是文本中含有一些奇怪的字符，比如 `<br /><br />`

我们要把这些脏东西去掉

---

### 解释

#### 数据清理和预处理

1. **导入正则表达式库和ClassLabel类**：
    - `re`库用于处理文本中的正则表达式操作。
    - `ClassLabel`类用于处理数据集中的标签，使其具有更好的可读性和易用性。

2. **定义文本清理函数`clean_text`**：
    - 该函数用于移除文本中的HTML标签。
    - 通过正则表达式替换，将`<br />`标签替换为空格，并移除其他HTML标签。

3. **定义数据预处理函数`preprocess_data`**：
    - 该函数对每个示例进行文本清理，并使用BERT分词器将文本转换为模型所需的输入格式。
    - 包括截断和填充，使所有输入文本的长度一致。

4. **定义标签编码函数`encode_labels`**：
    - 该函数将标签从字符串形式转换为整数形式。
    - 使用`ClassLabel`对象将标签转换为对应的整数值，便于模型进行分类任务。

5. **加载分词器和模型**：
    - 使用`BertTokenizer`加载预训练的BERT分词器，模型名称为`bert-base-uncased`。

6. **创建ClassLabel对象**：
    - 定义标签的类别及其对应的名称，即负面（negative）和正面（positive）。

7. **对数据集中的标签进行编码**：
    - 使用`dataset.map`方法，将标签转换为整数编码形式，便于后续的模型训练和评估。

In [3]:
import re
from datasets import ClassLabel

def clean_text(text):
    # 去除HTML标签
    text = re.sub(r'<br\s*/?>', ' ', text)
    # 移除其他HTML标签
    text = re.sub(r'<.*?>', '', text)
    return text

def preprocess_data(examples):
    texts = [clean_text(text) for text in examples['text']]
    return tokenizer(texts, truncation=True, padding='max_length', max_length=512)

# 将标签转换为分类名称
def encode_labels(example):
    example['label'] = class_labels.str2int(example['label'])
    return example

# 加载分词器
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)

# 创建ClassLabel对象
class_labels = ClassLabel(num_classes=2, names=['negative', 'positive'])

# 对数据集中的标签进行编码
encoded_dataset = dataset.map(encode_labels, batched=True)

---

### 解释

#### 数据预处理和格式设置

1. **预处理数据集**：

```python
encoded_dataset = encoded_dataset.map(preprocess_data, batched=True)
```
- **作用**：对整个数据集应用预处理函数`preprocess_data`。
- **原因**：将每个示例的文本清理并转换为BERT模型所需的输入格式（tokenizer输出）。
- **方法**：使用`map`方法批量处理数据集，`batched=True`表示一次处理多个示例，以提高处理效率。

2. **将数据集划分为训练集和验证集**：

```python
train_dataset = encoded_dataset['train']
test_dataset = encoded_dataset['test']
```
- **作用**：将预处理后的数据集划分为训练集和验证集。
- **原因**：在训练模型时需要有训练数据和验证数据，以评估模型的性能和防止过拟合。
- **方法**：直接从`encoded_dataset`中获取预先定义好的`train`和`test`数据集。

3. **移除不需要的列**：

```python
train_dataset = train_dataset.remove_columns(['text'])
test_dataset = test_dataset.remove_columns(['text'])
```
- **作用**：移除数据集中不再需要的原始文本列。
- **原因**：在模型训练和评估过程中，不需要原始的文本数据，只需要经过分词处理后的输入数据（如input_ids、attention_mask等）。
- **方法**：使用`remove_columns`方法从数据集中移除指定的列。

4. **设置格式**：

```python
train_dataset.set_format('torch')
test_dataset.set_format('torch')
```
- **作用**：将数据集格式设置为PyTorch张量。
- **原因**：BERT模型和训练过程使用的是PyTorch框架，因此需要将数据转换为PyTorch张量格式。
- **方法**：使用`set_format`方法将数据集格式设置为'torch'，即PyTorch张量格式。


In [5]:
# 预处理数据集
encoded_dataset = encoded_dataset.map(preprocess_data, batched=True)

# 将数据集划分为训练集和验证集
train_dataset = encoded_dataset['train']
test_dataset = encoded_dataset['test']

# 移除不需要的列
train_dataset = train_dataset.remove_columns(['text'])
test_dataset = test_dataset.remove_columns(['text'])

# 设置格式
train_dataset.set_format('torch')
test_dataset.set_format('torch')


---



#### 输出解释

```plaintext
Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 25000
})
```

- **Dataset**：这是一个包含处理后数据的对象。
- **features**：
  - `label`：整数形式的标签，用于表示文本的情感类别（例如，0表示负面，1表示正面）。
  - `input_ids`：经过BERT分词器处理后的文本输入ID序列。这些ID表示输入文本中每个词的标识符。
  - `token_type_ids`：用于区分句子A和句子B的ID。对于单个句子任务，通常全为0。
  - `attention_mask`：用于指示哪些token是实际的词汇（值为1），哪些是填充的（值为0）。
- **num_rows**：数据集中样本的数量。对于训练集和测试集，样本数量分别为25,000。

### 详细解释

1. **label**：
   - **作用**：表示文本的情感类别标签。
   - **原因**：在情感分类任务中，需要明确的标签来指导模型学习和预测。标签通常是整数形式，表示不同的类别。

2. **input_ids**：
   - **作用**：表示输入文本的词汇标识符序列。
   - **原因**：BERT模型需要输入ID序列来理解和处理文本。分词器将每个词转换为对应的ID，形成输入ID序列。

3. **token_type_ids**：
   - **作用**：区分不同句子。对于单句输入，所有值通常为0。
   - **原因**：在处理多个句子时（如问答任务），需要区分句子A和句子B。对于情感分类这样的单句任务，通常不需要区分，所有值为0。

4. **attention_mask**：
   - **作用**：指示哪些token是实际的词汇，哪些是填充的。
   - **原因**：BERT模型需要知道哪些部分是实际输入，哪些是填充，用于处理不同长度的输入文本。实际词汇的值为1，填充部分的值为0。

5. **num_rows**：
   - **作用**：表示数据集中样本的数量。
   - **原因**：明确数据集的规模，有助于了解训练和评估过程中将处理的数据量。

### 总结

通过这些信息，我们可以确认数据集已经成功预处理并转换为适合BERT模型输入的格式。预处理后的数据集包含以下特征：

- `label`：情感分类标签。
- `input_ids`：文本的输入ID序列。
- `token_type_ids`：用于区分句子的ID序列（对于单句任务，全为0）。
- `attention_mask`：指示哪些部分是实际输入，哪些是填充。

数据集中的样本数量为25,000，适用于后续的模型训练和评估。这个预处理步骤确保了数据的干净和一致性，并转换为BERT模型可以接受的格式。

In [6]:
# 查看预处理后的数据集信息
print(train_dataset)
print(test_dataset)

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 25000
})
Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 25000
})


In [7]:
# 检查预处理后的数据集字段
print("Train Dataset Features:")
print(train_dataset.features)

print("Test Dataset Features:")
print(test_dataset.features)

# 查看训练集中的前5个样本
print("\nFirst 5 samples in the Train Dataset:")
for i in range(5):
    print(f"Sample {i+1}:")
    print(f"Input IDs: {train_dataset[i]['input_ids']}")
    print(f"Attention Mask: {train_dataset[i]['attention_mask']}")
    print(f"Label: {train_dataset[i]['label']}")
    print()

# 查看测试集中的前5个样本
print("\nFirst 5 samples in the Test Dataset:")
for i in range(5):
    print(f"Sample {i+1}:")
    print(f"Input IDs: {test_dataset[i]['input_ids']}")
    print(f"Attention Mask: {test_dataset[i]['attention_mask']}")
    print(f"Label: {test_dataset[i]['label']}")
    print()


Train Dataset Features:
{'label': ClassLabel(names=['neg', 'pos'], id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}
Test Dataset Features:
{'label': ClassLabel(names=['neg', 'pos'], id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

First 5 samples in the Train Dataset:
Sample 1:
Input IDs: tensor([  101,  1045, 12524,  1045,  2572,  8025,  1011,  3756,  2013,  2026,
         2678,  3573,  2138,  1997,  2035,  1996,  6704,  2008,  5129,  2009,
         2043,  2009,  2001,  2034,  2207,  1999,  3476,  1012,  1045,  2036,
         2657,  2008,  2012,  2034,  2009,  2

In [8]:
from transformers import TrainingArguments
from transformers import Trainer

model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

training_args = TrainingArguments(
    output_dir='./results',          # 输出目录
    num_train_epochs=3,              # 训练轮次
    per_device_train_batch_size=8,   # 每个设备的训练批量大小
    per_device_eval_batch_size=8,    # 每个设备的评估批量大小
    warmup_steps=500,                # 预热步数
    weight_decay=0.01,               # 权重衰减
    logging_dir='./logs',            # 日志目录
    logging_steps=10,                # 日志记录步数
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [9]:
# 训练模型
trainer.train()

Step,Training Loss
10,0.685300
20,0.721400
30,0.702500
40,0.721600
50,0.669500
60,0.649200
70,0.657300
80,0.625500
90,0.635100
100,0.663700


TrainOutput(global_step=9375, training_loss=0.2018804952848951, metrics={'train_runtime': 3221.2788, 'train_samples_per_second': 23.283, 'train_steps_per_second': 2.91, 'total_flos': 1.9733329152e+16, 'train_loss': 0.2018804952848951, 'epoch': 3.0})

In [10]:
# 保存模型
output_dir = "./saved_model"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.txt',
 './saved_model/added_tokens.json')

In [11]:
# 评估模型
results = trainer.evaluate(eval_dataset=test_dataset)
print(results)

{'eval_loss': 0.3212610185146332, 'eval_runtime': 364.3962, 'eval_samples_per_second': 68.607, 'eval_steps_per_second': 8.576, 'epoch': 3.0}


In [ ]:

# # 预测
# predictions = trainer.predict(test_dataset)
# preds = np.argmax(predictions.predictions, axis=1)
# labels = predictions.label_ids


In [14]:
import pandas as pd

# 创建DataFrame
texts = [clean_text(text) for text in dataset['test']['text']]
df = pd.DataFrame({
    'Text': texts,
    'Actual Label': labels,
    'Predicted Label': preds
})

# 显示前20个样本
df_sample = df.sample(20)
print(df_sample)

                                                    Text  Actual Label  \
6868   I could not believe how terrible and boring th...             0   
24016  I rented Boogie Nights last week and I could t...             1   
9668   First off, this movie is not near complete, my...             0   
13640  I watched this mini in the early eighties. Sam...             1   
14018  This movie was never intended as a big-budget ...             1   
7488   I thought this was an extremely bad movie. The...             0   
5804   This was one of the biggest pieces of crap I h...             0   
12909  I just watched it for the second time today an...             1   
3386   I've read through a lot of the comments here a...             0   
9567   I usually much prefer French movies over Ameri...             0   
21423  This was very funny, even if it fell apart a l...             1   
3503   SPOILER!!!! Mind Ripper hmmmm.... I had just w...             0   
6657   Some folkie friends recommended

In [15]:
# 保存为CSV文件
df.to_csv('predictions.csv', index=False)

In [2]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix, classification_report
import pandas as pd

# 读取CSV文件
df = pd.read_csv('predictions.csv')

# 提取实际标签和预测标签
y_true = df['Actual Label']
y_pred = df['Predicted Label']

# 计算准确率、F1得分和召回率
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')

# 打印分类报告
print(classification_report(y_true, y_pred, target_names=['negative', 'positive']))

# 计算混淆矩阵
cm = confusion_matrix(y_true, y_pred)

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Recall: {recall}")
print(f"Confusion Matrix:\n{cm}")


              precision    recall  f1-score   support

    negative       0.94      0.94      0.94     12500
    positive       0.94      0.94      0.94     12500

    accuracy                           0.94     25000
   macro avg       0.94      0.94      0.94     25000
weighted avg       0.94      0.94      0.94     25000

Accuracy: 0.93996
F1 Score: 0.9399599576357461
Recall: 0.93996
Confusion Matrix:
[[11739   761]
 [  740 11760]]
